In [17]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics.cluster import normalized_mutual_info_score, adjusted_rand_score
from sentence_transformers import SentenceTransformer
import numpy as np

from sklearn.decomposition import FactorAnalysis # library used for AFC

from sklearn.cluster import KMeans # library used for AFC

'''
Variables: 
---------

corpus : list of documents
embeddings : documents embeddings of size NxM (N : number of documents, M : embedding dimension) 
red_emd : reduced embeddings matrix using dimentionality reduction
k : number of clusters
labels : documents labels
pred : list of clustering predicted clusters 

''';

In [18]:
def dim_red(mat, p):
    '''
    Perform dimensionality reduction using Factor Analysis (l'AFC)

    Input:
    -----
        mat : NxM list or array-like 
        p : number of dimensions to keep 
    Output:
    ------
        red_mat : NxP array such that p<<m
    '''
    # Convert the input list to a NumPy array if it's not already an array
    mat_np = mat if isinstance(mat, np.ndarray) else np.array(mat)
    
    # Initialize Factor Analysis model with desired number of components
    fa = FactorAnalysis(n_components=p)
    
    # Fit and transform the data to the lower-dimensional space
    red_mat = fa.fit_transform(mat_np)
    
    return red_mat


In [19]:
def clust(mat, k):
    '''
    Perform clustering using KMeans

    Input:
    -----
        mat : input list or array-like
        k : number of clusters
    Output:
    ------
        pred : list of predicted labels
    '''
    # Convert the input list to a NumPy array if it's not already an array
    mat_np = mat if isinstance(mat, np.ndarray) else np.array(mat)
    
    # Initialize KMeans model with the desired number of clusters
    kmeans = KMeans(n_clusters=k)
    
    # Fit KMeans to the data and predict cluster labels
    pred = kmeans.fit_predict(mat_np)
    
    return pred.tolist()


In [20]:
# import data
ng20 = fetch_20newsgroups(subset='test')
corpus = ng20.data[:2000]
labels = ng20.target[:2000]
k = len(set(labels))

# embedding
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
embeddings = model.encode(corpus)

# perform dimentionality reduction
red_emb = dim_red(embeddings, 20)

# perform clustering
pred = clust(red_emb, k)

# evaluate clustering results
nmi_score = normalized_mutual_info_score(pred,labels)
ari_score = adjusted_rand_score(pred,labels)

print(f'NMI: {nmi_score:.2f} \nARI: {ari_score:.2f}')


C:\Users\moham\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\moham\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=8.
  warnings.warn(


NMI: 0.40 
ARI: 0.23
